<a href="https://colab.research.google.com/github/PanoEvJ/summarization_RLHF/blob/main/T5_base_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

!pip install -q transformers==4.30
!pip install -q -U sentencepiece
!pip install -q huggingface_hub
!pip install -q tdqm torch>=0.3.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import os
import random
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset, Dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [5]:
from datasets import get_dataset_split_names

dataset_name = 'CarperAI/openai_summarize_comparisons'
get_dataset_split_names(dataset_name)


['train', 'test', 'valid1', 'valid2']

In [6]:
dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='valid1')

dataset.info.features

{'prompt': Value(dtype='string', id=None),
 'chosen': Value(dtype='string', id=None),
 'rejected': Value(dtype='string', id=None)}

In [7]:
train_dict = {'prompt': [], 'response':[]}
# eval_dict = {'prompt': [], 'response':[]}

# shuffled_dataset = dataset.shuffle(seed=42)
# for i in range(0, len(shuffled_dataset), 2):
for i in range(len(dataset)):
  # if random.randint(1,2)==1:
  #   key = 'chosen'
  # else:
  #   key = 'rejected'
  train_dict["prompt"].append(dataset[i]["prompt"])
  train_dict["response"].append(dataset[i]['chosen'])
  # train_dict["prompt"].append(shuffled_dataset[i]["prompt"])
  # train_dict["response"].append(shuffled_dataset[i][key])
  # eval_dict["prompt"].append(shuffled_dataset[i+1]["prompt"])
  # eval_dict["response"].append(shuffled_dataset[i+1][key])

train_dataset = Dataset.from_dict(train_dict)
# eval_dataset =  Dataset.from_dict(eval_dict)

In [8]:
model_id = "t5-base"

In [9]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    use_cache=False,
    # load_in_8bit=True,
    device_map="auto"
)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(base_model)

trainable params: 222903552 || all params: 222903552 || trainable%: 100.0


In [ ]:
# see the available modules by printint out the model
print(base_model)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          model_max_length=512,
                                          truncation=True,
                                          padding=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

In [13]:
training_args = TrainingArguments(   # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
    output_dir='./output/',
    # save_steps=5000,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-4,
    logging_steps=500,
    optim='adamw_torch',
    # bf16=True,
    # remove_unused_columns=False,
    run_name="sft_t5_base",
    # logging_first_step=True,
    # max_steps=50
)

In [15]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="response",
    max_seq_length=512
)
trainer.train()
trainer.save_model('./output/')

Map:   0%|          | 0/33082 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.015200
1000,0.007300
1500,0.008700
2000,0.008400
2500,0.006300
3000,0.005900
3500,0.003000
4000,0.003600
4500,0.002700
5000,0.003400


In [16]:
trainer.model.push_to_hub('PanoEvJ/T5_base_SFT_summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'generation_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'pytorch_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/PanoEvJ/T5_base_SFT_summarization/commit/582ad7a26606459fe96041c0485f31d76097f8bc', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='582ad7a26606459fe96041c0485f31d76097f8bc', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
trainer.tokenizer.push_to_hub('PanoEvJ/T5_base_SFT_summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'special_tokens_map.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'spiece.model'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/PanoEvJ/T5_base_SFT_summarization/commit/547405800f4e7e7cb5eb6724927f3f04d482ccde', commit_message='Upload tokenizer', commit_description='', oid='547405800f4e7e7cb5eb6724927f3f04d482ccde', pr_url=None, pr_revision=None, pr_num=None)